In [1]:
import os
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal
from skimage import feature
from bigfish import detection

In [2]:
# Assumes the folders are present in the CWD
path_input = "./img_Xenium"
path_output = "./output_Xenium"
path_figures = "./figures_Xenium"
path_fwhm = "./fwhm_Xenium"

paths = os.listdir(path_input)
paths = [p for p in paths if p.endswith("tif")]
print(paths)

['_morphology_X19935_Y7267_W471_H471.ome.tif']


In [3]:
np.int = int # Fixes compute_snr_spots using the now deprecated numpy.int https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [4]:
RADIUS_FACTOR = 2.0
PIXEL_SIZE = (212.5 , 212.5)  # in nanometer (one value per dimension yx)
SPOT_RADIUS = (212.5 * RADIUS_FACTOR, 212.5 * RADIUS_FACTOR)  # in nanometer (one value per dimension yx)
THRESHOLD_LOW = 100
MIN_DIST= 1

In [5]:
# Identify peaks as local maxima  and calculate signal to noise ratio with big-fish (https://big-fish.readthedocs.io/en/stable/detection/spots.html#compute-signal-to-noise-ratio)
summary = []
for path in paths:
    img_path = os.path.join(path_input, path)
    img = tifffile.imread(img_path)[5,:,:] # Exclude out of focus z-slices
    clean_img = np.copy(img)
    clean_img[clean_img < THRESHOLD_LOW] = 0 # Remove all pixels < THRESHOLD_LOW
    spots = feature.peak_local_max(clean_img, min_distance=MIN_DIST,
                                   threshold_abs=None,
                                   footprint = None,
                                   threshold_rel=None,
                                   exclude_border=False) # Identify peaks as local maxima
    output_path = os.path.join(path_output, path.replace(".ome.tif", ".csv"))
    df = pd.DataFrame(spots)
    df.to_csv(path_or_buf = output_path, sep=',', na_rep='', float_format=None, columns=None, header=True, index=False)

    snr = detection.compute_snr_spots(img, spots[:, 0:2], PIXEL_SIZE, SPOT_RADIUS)

    results = {"img_path":img_path, "spots_path":output_path, "spots":spots.shape[0], "snr":snr, "threshold_low":THRESHOLD_LOW}
    summary.append(results)

# Save and print the summary
summary = pd.DataFrame(summary)
print(summary)
summary.to_csv(path_or_buf = os.path.join(path_output, "summary.csv"), sep=',', na_rep='', float_format=None, columns=None, header=True, index=False)   

                                            img_path  \
0  ./img_Xenium/_morphology_X19935_Y7267_W471_H47...   

                                          spots_path  spots     snr  \
0  ./output_Xenium/_morphology_X19935_Y7267_W471_...   3158  4.7263   

   threshold_low  
0            100  


In [ ]:
# Plot the identified peaks on the image
for path in paths:
    img_path = os.path.join(path_input, path)
    tbl_path = os.path.join(path_output, path.replace(".ome.tif", ".csv"))
    out_path = os.path.join(path_figures, path.replace(".ome.tif", "-figure.png"))
  
    img = tifffile.imread(img_path)[5,:,:] # Exclude out of focus z-slices
    
    data = pd.read_csv(tbl_path)
    data.rename(columns = {"0":"y", "1":"x"}, inplace = True)
    data.loc[:, "y"] = img.shape[0] - data.loc[:, "y"]
    plt.clf()
    plt.set_cmap('hot')
    f, ax = plt.subplots(figsize=(36, 36))
    ax.imshow(img, extent=[0, img.shape[1], 0, img.shape[0]], aspect='auto')
    sns.scatterplot(x="x", y="y", facecolors = 'none', edgecolors = 'lime',
                data=data, ax=ax)
    ax.set_axis_off()
    f.savefig(out_path, bbox_inches='tight', pad_inches = 0)
    print((img_path, tbl_path, out_path))

In [7]:
# Calculate FWHM using scipy.signal.peak_widths (https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.peak_widths.html)
header = ["y_px", "x_px", "y_nm", "x_nm", "fwhm_y_px", "fwhm_x_px", "fwhm_y_nm", "fwhm_x_nm"]
for path in paths:
    tbl_path = os.path.join(path_output, path.replace(".ome.tif", ".csv"))
    out_path = os.path.join(path_fwhm, path.replace(".csv", "-fwhm.csv"))
    img_path = os.path.join(path_input, path)
    img = tifffile.imread(img_path)[5,:] # Exclude out of focus z-slices

    peaks = pd.read_csv(tbl_path)
    peaks = np.asarray(peaks)
    results = np.concatenate((peaks, np.full((peaks.shape[0], 6), -1.0)), axis = 1)
    
    for i,p in enumerate(peaks):
         results[i,4] = signal.peak_widths(img[: ,p[1]], np.full((1), p[0]), rel_height=0.5, prominence_data=None, wlen=None)[0] # Y
         results[i,5] = signal.peak_widths(img[p[0], :,], np.full((1), p[1]), rel_height=0.5, prominence_data=None, wlen=None)[0] # X
    results[:, 2] = results[:, 0] * PIXEL_SIZE[0]  # Peak coordinate Y 
    results[:, 3] = results[:, 1] * PIXEL_SIZE[1]  # Peak coordinate X
    results[:, 6] = results[:, 4] * PIXEL_SIZE[0]  # Peak FWHM Y
    results[:, 7] = results[:, 5] * PIXEL_SIZE[1]  # Peak FWHM X
    
    df = pd.DataFrame(results, columns=header)
    df.to_csv(path_or_buf = out_path, sep=',', na_rep='',
              float_format=None, columns=None, header=True, index=False)


/tmp/ipykernel_69304/252985659.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[i,4] = signal.peak_widths(img[: ,p[1]], np.full((1), p[0]), rel_height=0.5, prominence_data=None, wlen=None)[0] # Y
/tmp/ipykernel_69304/252985659.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[i,5] = signal.peak_widths(img[p[0], :,], np.full((1), p[1]), rel_height=0.5, prominence_data=None, wlen=None)[0] # X
/tmp/ipykernel_69304/252985659.py:15: PeakPropertyWarning: some peaks have a prominence of 0
  results[i,5] = signal.peak_widths(img[p[0], :,], np.full((1), p[1]), rel_height=0.5, prominence_data=None, wlen=None)[0] # X
/tmp/ipyk

./output_Xenium/_morphology_X19935_Y7267_W471_H471
